In [1]:
import pandas as pd
import psycopg2

CONNECT_DB = "host=localhost port=5432 dbname=university_db user=dataengineer password=secret"

In [22]:
import pandas as pd
import psycopg2
from psycopg2.extras import RealDictCursor

# Connection string
CONNECT_DB = "host=localhost port=5432 dbname=university_db user=dataengineer password=secret"

def run_query(query):
    """
    Execute a SQL query and return the result as a DataFrame.
    """
    try:
        # Establish connection
        conn = psycopg2.connect(CONNECT_DB)
        
        # Execute query and fetch data
        with conn.cursor(cursor_factory=RealDictCursor) as cursor:
            cursor.execute(query)
            data = cursor.fetchall()
        
        # Convert fetched data to DataFrame
        df = pd.DataFrame(data)
        return df
    
    except Exception as e:
        print(f"Error: {e}")
    
    finally:
        # Close connection
        if conn:
            conn.close()


In [29]:
query = '''
WITH attendances_data AS (
SELECT *
FROM wh__enrollments AS e
FULL OUTER JOIN wh__schedules AS s
    ON e.schedule_id = s.id
RIGHT JOIN wh__attendances AS a
    ON e.student_id = a.student_id
    AND e.schedule_id = a.schedule_id
)

SELECT *, name
FROM attendances_data AS a
LEFT JOIN wh__courses AS c
ON a.course_id = c.id;
'''

result_df = run_query(query)
result_df

,id,student_id,schedule_id,academic_year,semester,enroll_dt,course_id,lecturer_id,start_dt,end_dt,course_day,schedule_date,week_number,attend_dt,name
0,3,1,2,2019/2020,1,2019-08-17,3,4,2019-09-09,2019-12-08,3,2019-09-10,1,2019-09-11,Data Mining
1,3,1,2,2019/2020,1,2019-08-17,3,4,2019-09-09,2019-12-08,3,2019-09-10,1,2019-10-15,Data Mining
2,3,1,2,2019/2020,1,2019-08-17,3,4,2019-09-09,2019-12-08,3,2019-09-10,1,2019-10-09,Data Mining
3,3,1,2,2019/2020,1,2019-08-17,3,4,2019-09-09,2019-12-08,3,2019-09-10,1,2019-10-08,Data Mining
4,3,1,2,2019/2020,1,2019-08-17,3,4,2019-09-09,2019-12-08,3,2019-09-10,1,2019-10-02,Data Mining
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,3,13,10,2019/2020,2,2019-08-17,3,37,2020-01-27,2020-04-26,5,2020-01-30,1,2020-03-27,Data Mining
376,3,13,10,2019/2020,2,2019-08-17,3,37,2020-01-27,2020-04-26,5,2020-01-30,1,2020-04-03,Data Mining
377,3,13,10,2019/2020,2,2019-08-17,3,37,2020-01-27,2020-04-26,5,2020-01-30,1,2020-04-10,Data Mining
378,3,13,10,2019/2020,2,2019-08-17,3,37,2020-01-27,2020-04-26,5,2020-01-30,1,2020-04-17,Data Mining


In [140]:
query = '''
SELECT *
FROM stg__schedules;
'''

result_df = run_query(query)
df = result_df.copy()
df

,id,course_id,lecturer_id,start_dt,end_dt,course_days
0,1,1,11,09-Sep-19,08-Dec-19,"2,4,6"
1,2,3,4,09-Sep-19,08-Dec-19,"3,4"
2,3,4,13,09-Sep-19,08-Dec-19,"2,5"
3,4,5,28,09-Sep-19,08-Dec-19,3
4,5,2,56,27-Jan-20,26-Apr-20,"2,4"
5,6,7,1,27-Jan-20,26-Apr-20,"4,5"
6,7,1,11,27-Jan-20,26-Apr-20,"2,4,6"
7,8,7,22,27-Jan-20,26-Apr-20,"2,3"
8,9,8,56,27-Jan-20,26-Apr-20,5
9,10,3,37,27-Jan-20,26-Apr-20,"5,6"


In [137]:
df['start_dt'] = pd.to_datetime(df['start_dt'], format="%d-%b-%y", errors='coerce')
df['end_dt'] = pd.to_datetime(df['end_dt'], format="%d-%b-%y", errors='coerce')

from datetime import datetime, timedelta

all_dates = []

for _, row in df.iterrows():
    start_date = row['start_dt']
    end_date = row['end_dt']
    days_of_week = [int(day) for day in row['course_days'].split(',')]

    current_date = start_date
    while current_date <= end_date:
        if current_date.weekday() + 1 in days_of_week:
            course_date = current_date.weekday() + 1
            adjusted_date = current_date - timedelta(days=1)
            all_dates.append({
                'course_id': row['course_id'],
                'lecturer_id': row['lecturer_id'],
                'start_dt': row['start_dt'],
                'end_dt': row['end_dt'],
                'course_day': course_date,
                'schedule_date': (adjusted_date).strftime('%Y-%m-%d'),
                'week_number': ((current_date - start_date).days // 7) + 1
            })
        current_date += timedelta(days=1)

df = pd.DataFrame(all_dates)
df.reset_index(drop=True, inplace=True)
df.index += 1
df.index.name = 'id'
df.reset_index(inplace=True)
df.to_csv('res.csv')

In [143]:
from datetime import datetime

def is_course_day(start_dt, days_of_week):
    # Convert 'start_dt' from string to a datetime object
    start_date = datetime.strptime(start_dt, '%d-%b-%Y')
    
    # Get the weekday (0=Monday, 1=Tuesday, ..., 6=Sunday)
    weekday = start_date.weekday()
    
    # Map weekday to your custom week starting with Sunday
    custom_weekday = (weekday + 1) % 7 + 1
    
    return custom_weekday in days_of_week

# Example usage
start_dt = '09-Sep-2019'  # Monday
days_of_week = {2}  # Monday, Wednesday, Friday
print(is_course_day(start_dt, days_of_week))  # Should return True for Monday


True


In [26]:
query = """
WITH weekly_attendance AS (
    SELECT
        e.semester,
        s.week_number AS week_id,
        c.name AS course_name,
        COUNT(DISTINCT a.student_id) AS attendance_count,
        COUNT(DISTINCT e.student_id) AS total_enrollments
    FROM
        wh__attendances a
        JOIN wh__enrollments e ON a.schedule_id = e.schedule_id AND a.student_id = e.student_id
        JOIN wh__schedules s ON e.schedule_id = s.id
        JOIN wh__courses c ON s.course_id = c.id
    GROUP BY
        e.semester,
        s.week_number,
        c.name
)
SELECT
    semester AS semester_id,
    week_id,
    course_name,
    (attendance_count::FLOAT / total_enrollments) * 100 AS attendance_pct
FROM
    weekly_attendance
ORDER BY
    semester_id,
    week_id,
    course_name;
"""

result_df = run_query(query)
result_df

,semester_id,week_id,course_name,attendance_pct
0,1,1,Applied Data Science,100.0
1,1,1,Data Mining,100.0
2,1,1,Introduction to Applied Machine Learning,100.0
3,1,1,Statistic,100.0
4,2,1,Data Mining,100.0
5,2,1,Introduction to Applied Machine Learning,100.0
6,2,1,Natural Language Processing,100.0
7,2,1,Python Programming,100.0


In [32]:
query = """
WITH weekly_attendance AS (
    SELECT
        e.semester,
        s.week_number AS week_id,
        c.name AS course_name,
        COUNT(DISTINCT a.student_id) AS attendance_count,
        COUNT(DISTINCT e.student_id) AS total_enrollments
    FROM
        wh__attendances a
        FULL OUTER JOIN wh__enrollments e ON a.schedule_id = e.schedule_id AND a.student_id = e.student_id
        FULL OUTER JOIN wh__schedules s ON e.schedule_id = s.id
        LEFT JOIN wh__courses c ON s.course_id = c.id
    GROUP BY
        e.semester,
        s.week_number,
        c.name
)
SELECT *
FROM
    weekly_attendance
"""

result_df = run_query(query)
result_df

,semester,week_id,course_name,attendance_count,total_enrollments
0,1.0,1,Applied Data Science,2,2
1,1.0,1,Data Mining,3,3
2,1.0,1,Introduction to Applied Machine Learning,2,2
3,1.0,1,Statistic,2,2
4,2.0,1,Data Mining,2,2
...,...,...,...,...,...
94,NaN,13,Information Retrieval,0,0
95,NaN,13,Introduction to Applied Machine Learning,0,0
96,NaN,13,Natural Language Processing,0,0
97,NaN,13,Python Programming,0,0


In [39]:
query = '''
SELECT schedule_id, semester, COUNT(student_id)
FROM wh__enrollments
GROUP BY schedule_id, semester
ORDER BY semester, schedule_id

'''

result_df = run_query(query)
result_df

,schedule_id,semester,count
0,1,1,2
1,2,1,3
2,3,1,2
3,4,1,2
4,5,2,1
5,7,2,2
6,8,2,1
7,10,2,2


In [38]:
query = '''
SELECT attend_dt, schedule_id, COUNT(student_id)
FROM wh__attendances
GROUP BY schedule_id, attend_dt
'''

result_df = run_query(query)
result_df

,attend_dt,schedule_id,count
0,2019-11-01,1,2
1,2020-02-12,7,2
2,2019-11-19,2,3
3,2020-04-03,10,2
4,2019-10-21,1,2
...,...,...,...
209,2020-03-26,10,2
210,2019-10-25,1,1
211,2019-10-07,1,2
212,2020-03-02,7,2


In [71]:
query = '''
SELECT course_id, schedule_date, week_number --, COUNT(course_day) AS num_course
FROM wh__schedules
--GROUP BY course_id, schedule_date, week_number
--ORDER BY COUNT(course_day)
'''

result_df = run_query(query)
result_df

,course_id,schedule_date,week_number
0,1,2019-09-09,1
1,3,2019-09-10,1
2,4,2019-09-09,1
3,5,2019-09-10,1
4,2,2020-01-27,1
...,...,...,...
255,3,2020-04-10,11
256,3,2020-04-16,12
257,3,2020-04-17,12
258,3,2020-04-23,13


In [139]:
query = '''
WITH enrollment_num AS (
    SELECT schedule_id, semester, COUNT(student_id) AS student_enr
    FROM wh__enrollments
    GROUP BY schedule_id, semester
    ORDER BY semester, schedule_id
),
schedule_sum AS (
    SELECT course_id, schedule_date, week_number
    FROM wh__schedules
),
attendance_sum AS (
    SELECT attend_dt, schedule_id, COUNT(student_id) AS student_atd
    FROM wh__attendances
    GROUP BY schedule_id, attend_dt
)
SELECT 
    COALESCE(a.schedule_id, s.course_id),
    attend_dt,
    schedule_date,
    week_number
FROM attendance_sum AS a
    FULL OUTER JOIN schedule_sum AS s
        ON a.schedule_id = s.course_id
        AND a.attend_dt = s.schedule_date
ORDER BY schedule_id, attend_dt
'''
result_df = run_query(query)
result_df

,coalesce,attend_dt,schedule_date,week_number
0,1,2019-09-09,2019-09-09,1.0
1,1,2019-09-11,None,NaN
2,1,2019-09-13,None,NaN
3,1,2019-09-16,None,NaN
4,1,2019-09-18,None,NaN
...,...,...,...,...
413,1,None,2020-04-03,10.0
414,7,None,2020-03-24,9.0
415,1,None,2020-04-01,10.0
416,7,None,2020-03-31,10.0


In [76]:
query = '''
WITH enrollment_num AS (
    SELECT schedule_id, semester, COUNT(student_id) AS student_enr
    FROM wh__enrollments
    GROUP BY schedule_id, semester
    ORDER BY semester, schedule_id
),
schedule_sum AS (
    SELECT course_id, schedule_date, week_number
    FROM wh__schedules
),
attendance_sum AS (
    SELECT attend_dt, schedule_id, COUNT(student_id) AS student_atd
    FROM wh__attendances
    GROUP BY schedule_id, attend_dt
),
attendance_pct AS (
    SELECT
        s.course_id,
        week_number,
        s.schedule_date,
        a.attend_dt,
        COALESCE(student_atd, 0) AS num_student_atd
    FROM schedule_sum AS s
        LEFT JOIN attendance_sum AS a
            ON s.schedule_date = a.attend_dt
            AND s.course_id = a.schedule_id
)
SELECT
    course_id,
    week_number,
    COUNT(schedule_date) AS num_class,
    SUM(num_student_atd) AS student_attend
FROM attendance_pct
GROUP BY course_id, week_number
ORDER BY course_id, week_number
'''

result_df = run_query(query)
result_df

,course_id,week_number,num_class,student_attend
0,1,1,5,2
1,1,2,3,0
2,1,3,3,0
3,1,4,5,2
4,1,5,6,6
...,...,...,...,...
86,8,9,1,0
87,8,10,1,0
88,8,11,1,0
89,8,12,1,0
